# Normalizer

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/normalizer](https://github.com/huseinzol05/Malaya/tree/master/example/normalizer).
    
</div>

In [1]:
%%time
import malaya

CPU times: user 6.32 s, sys: 1.43 s, total: 7.75 s
Wall time: 9.57 s


In [2]:
string1 = 'xjdi ke, y u xsuke makan HUSEIN kt situ tmpt, i hate it. pelikle, pada'
string2 = 'i mmg2 xske mknn HUSEIN kampng tmpat, i love them. pelikle saye'
string3 = 'perdana menteri ke11 sgt suka makn ayam, harganya cuma rm15.50'
string4 = 'pada 10/4, kementerian mengumumkan, 1/100'
string5 = 'Husein Zolkepli dapat tempat ke-12 lumba lari hari ni'
string6 = 'Husein Zolkepli (2011 - 2019) adalah ketua kampng di kedah sekolah King Edward ke-IV'
string7 = '2jam 30 minit aku tunggu kau, 60.1 kg kau ni, suhu harini 31.2c, aku dahaga minum 600ml'

### Load normalizer

This normalizer can load any spelling correction model, eg, `malaya.spell.probability`, or `malaya.spell.transformer`.

```python
def normalizer(speller = None, **kwargs):
    """
    Load a Normalizer using any spelling correction model.

    Parameters
    ----------
    speller : spelling correction object, optional (default = None)

    Returns
    -------
    result: malaya.normalize.Normalizer class
    """
```

In [3]:
corrector = malaya.spell.probability()
normalizer = malaya.normalize.normalizer(corrector)

#### normalize

```python
def normalize(
    self,
    string: str,
    normalize_text: bool = True,
    normalize_entity: bool = True,
    normalize_url: bool = False,
    normalize_email: bool = False,
    normalize_year: bool = True,
    normalize_telephone: bool = True,
    normalize_date: bool = True,
    normalize_time: bool = True,
    check_english_func: Callable = is_english,
    check_malay_func: Callable = is_malay,
):
    """
    Normalize a string.

    Parameters
    ----------
    string : str
    normalize_text: bool, (default=True)
        if True, will try to replace shortforms with internal corpus.
    normalize_entity: bool, (default=True)
        normalize entities, only effect `date`, `datetime`, `time` and `money` patterns string only.
    normalize_url: bool, (default=False)
        if True, replace `://` with empty and `.` with `dot`.
        `https://huseinhouse.com` -> `https huseinhouse dot com`.
    normalize_email: bool, (default=False)
        if True, replace `@` with `di`, `.` with `dot`.
        `husein.zol05@gmail.com` -> `husein dot zol kosong lima di gmail dot com`.
    normalize_year: bool, (default=True)
        if True, `tahun 1987` -> `tahun sembilan belas lapan puluh tujuh`.
        if True, `1970-an` -> `sembilan belas tujuh puluh an`.
        if False, `tahun 1987` -> `tahun seribu sembilan ratus lapan puluh tujuh`.
    normalize_telephone: bool, (default=True)
        if True, `no 012-1234567` -> `no kosong satu dua, satu dua tiga empat lima enam tujuh`
    normalize_date: bool, (default=True)
        if True, `01/12/2001` -> `satu disember dua ribu satu`.
    normalize_time: bool, (default=True)
        if True, `pukul 2.30` -> `pukul dua tiga puluh minit`.
    check_english_func: Callable, (default=malaya.text.is_english)
        function to check a word in english dictionary, default is malaya.text.is_english.
    check_malay_func: Callable, (default=malaya.text.is_malay)
        function to check a word in malay dictionary, default is malaya.text.is_malay.

    Returns
    -------
    string: {'normalize', 'date', 'money'}
    """
```

In [4]:
string = 'boleh dtg 8pagi esok tak atau minggu depan? 2 oktober 2019 2pm, tlong bayar rm 3.2k sekali tau'

In [5]:
normalizer.normalize(string)

{'normalize': 'boleh datang pukul lapan esok tidak atau minggu depan ? dua oktober dua ribu sembilan belas pukul empat belas , tolong bayar tiga ribu dua ratus ringgit sekali tahu',
 'date': {'8AM esok': datetime.datetime(2022, 4, 26, 8, 0),
  '2 oktober 2019 2pm': datetime.datetime(2019, 10, 2, 14, 0),
  'minggu depan': datetime.datetime(2022, 5, 2, 8, 16, 30, 892366)},
 'money': {'rm 3.2k': 'RM3200.0'}}

In [6]:
normalizer.normalize(string, normalize_entity = False)

{'normalize': 'boleh datang pukul lapan esok tidak atau minggu depan ? dua oktober dua ribu sembilan belas pukul empat belas , tolong bayar tiga ribu dua ratus ringgit sekali tahu',
 'date': {},
 'money': {}}

Here you can see, Malaya normalizer will normalize `minggu depan` to datetime object, also `3.2k ringgit` to `RM3200`

In [7]:
print(normalizer.normalize(string1))
print(normalizer.normalize(string2))
print(normalizer.normalize(string3))
print(normalizer.normalize(string4))
print(normalizer.normalize(string5))
print(normalizer.normalize(string6))
print(normalizer.normalize(string7))

{'normalize': 'tak jadi ke , kenapa awak tak suka makan HUSEIN kat situ tempat , saya hate it . pelik lah , pada', 'date': {}, 'money': {}}
{'normalize': 'saya memang-memang tak suka makan HUSEIN kampung tempat , saya love them . pelik lah saya', 'date': {}, 'money': {}}
{'normalize': 'perdana menteri kesebelas sangat suka makan ayam , harganya cuma lima belas ringgit lima puluh sen', 'date': {}, 'money': {'rm15.50': 'RM15.50'}}
{'normalize': 'pada sepuluh hari bulan empat , kementerian mengumumkan , satu per seratus', 'date': {}, 'money': {}}
{'normalize': 'Husein Zolkepli dapat tempat kedua belas lumba lari hari ini', 'date': {}, 'money': {}}
{'normalize': 'Husein Zolkepli ( dua ribu sebelas hingga dua ribu sembilan belas ) adalah ketua kampung di kedah sekolah King Edward keempat', 'date': {}, 'money': {}}
{'normalize': 'dua jam tiga puluh minit aku tunggu kamu , enam puluh perpuluhan satu kilogram kamu ini , suhu hari ini tiga puluh satu perpuluhan dua celsius , aku dahaga minum en

### Skip spelling correction

Simply pass `None` to `speller` to `normalizer = malaya.normalize.normalizer`. By default it is `None`.

In [8]:
normalizer = malaya.normalize.normalizer(corrector)
without_corrector_normalizer = malaya.normalize.normalizer(None)

In [9]:
normalizer.normalize(string2)

{'normalize': 'saya memang-memang tak suka makan HUSEIN kampung tempat , saya love them . pelik lah saya',
 'date': {},
 'money': {}}

In [10]:
without_corrector_normalizer.normalize(string2)

{'normalize': 'saya memang-memang tak suka mknn HUSEIN kampng tmpat , saya love them . pelik lah saya',
 'date': {},
 'money': {}}

### Pass kwargs preprocessing

Let say you want to skip to normalize date pattern, you can pass kwargs to normalizer, check original tokenizer implementation at https://github.com/huseinzol05/Malaya/blob/master/malaya/preprocessing.py#L103

In [11]:
normalizer = malaya.normalize.normalizer(corrector)
skip_date_normalizer = malaya.normalize.normalizer(corrector, date = False)

In [12]:
normalizer.normalize('tarikh program tersebut 14 mei')

{'normalize': 'tarikh program tersebut empat belas mei dua ribu dua puluh dua',
 'date': {'14 mei': datetime.datetime(2022, 5, 14, 0, 0)},
 'money': {}}

In [13]:
skip_date_normalizer.normalize('tarikh program tersebut 14 mei')

{'normalize': 'tarikh program tersebut empat belas mei',
 'date': {'14 mei': datetime.datetime(2022, 5, 14, 0, 0)},
 'money': {}}

### Normalize url

Let say you have an `url` word, example, `https://huseinhouse.com`, this parameter going to,

1. replace `://` with empty string.
2. replace `.` with ` dot `.
3. replace digits with string representation.
4. Capitalize `https`, `http`, and `www`.

Simply `normalizer.normalize(string, normalize_url = True)`, default is `False`.

In [14]:
normalizer = malaya.normalize.normalizer()
normalizer.normalize('web saya ialah https://huseinhouse.com')

{'normalize': 'web saya ialah https://huseinhouse.com',
 'date': {},
 'money': {}}

In [15]:
normalizer.normalize('web saya ialah https://huseinhouse.com', normalize_url = True)

{'normalize': 'web saya ialah HTTPS huseinhouse dot com',
 'date': {},
 'money': {}}

In [16]:
normalizer.normalize('web saya ialah https://huseinhouse02934.com', normalize_url = True)

{'normalize': 'web saya ialah HTTPS huseinhouse kosong dua sembilan tiga empat dot com',
 'date': {},
 'money': {}}

### Normalize email

Let say you have an `email` word, example, `husein.zol05@gmail.com`, this parameter going to,

1. replace `://` with empty string.
2. replace `.` with ` dot `.
3. replace `@` with ` di `.
4. replace digits with string representation.

Simply `normalizer.normalize(string, normalize_email = True)`, default is `False`.

In [17]:
normalizer = malaya.normalize.normalizer()
normalizer.normalize('email saya ialah husein.zol05@gmail.com')

{'normalize': 'email saya ialah husein.zol05@gmail.com',
 'date': {},
 'money': {}}

In [18]:
normalizer = malaya.normalize.normalizer()
normalizer.normalize('email saya ialah husein.zol05@gmail.com', normalize_email = True)

{'normalize': 'email saya ialah husein dot zol kosong lima di gmail dot com',
 'date': {},
 'money': {}}

### Normalize year

1. if True, `tahun 1987` -> `tahun sembilan belas lapan puluh tujuh`.
2. if True, `1970-an` -> `sembilan belas tujuh puluh an`.
3. if False, `tahun 1987` -> `tahun seribu sembilan ratus lapan puluh tujuh`.

Simply `normalizer.normalize(string, normalize_year = True)`, default is `True`.

In [19]:
normalizer = malaya.normalize.normalizer()

In [20]:
normalizer.normalize('$400 pada tahun 1998 berbanding lebih $1000')

{'normalize': 'empat ratus dollar pada tahun sembilan belas sembilan puluh lapan berbanding lebih seribu dollar',
 'date': {},
 'money': {'$400': '$400', '$1000': '$1000'}}

In [21]:
normalizer.normalize('$400 pada 1970-an berbanding lebih $1000')

{'normalize': 'empat ratus dollar pada sembilan belas tujuh puluhan berbanding lebih seribu dollar',
 'date': {},
 'money': {'$400': '$400', '$1000': '$1000'}}

In [22]:
normalizer.normalize('$400 pada tahun 1970-an berbanding lebih $1000')

{'normalize': 'empat ratus dollar pada tahun sembilan belas tujuh puluhan berbanding lebih seribu dollar',
 'date': {},
 'money': {'$400': '$400', '$1000': '$1000'}}

In [23]:
normalizer.normalize('$400 pada tahun 1998 berbanding lebih $1000', normalize_year = False)

{'normalize': 'empat ratus dollar pada tahun seribu sembilan ratus sembilan puluh lapan berbanding lebih seribu dollar',
 'date': {},
 'money': {'$400': '$400', '$1000': '$1000'}}

### Normalize telephone

1. if True, `no 012-1234567` -> `no kosong satu dua, satu dua tiga empat lima enam tujuh`.

Simply `normalizer.normalize(string, normalize_telephone = True)`, default is `True`.

In [24]:
normalizer = malaya.normalize.normalizer()

In [25]:
normalizer.normalize('no saya 012-1234567')

{'normalize': 'no saya kosong satu dua, satu dua tiga empat lima enam tujuh',
 'date': {},
 'money': {}}

In [26]:
normalizer.normalize('no saya 012-1234567', normalize_telephone = False)

{'normalize': 'no saya 012-1234567', 'date': {}, 'money': {}}

### Normalize date

1. if True, `01/12/2001` -> `satu disember dua ribu satu`.

Simply `normalizer.normalize(string, normalize_date = True)`, default is `True`.

In [27]:
normalizer = malaya.normalize.normalizer()
normalizer.normalize('saya akan gerak pada 1/11/2021')

{'normalize': 'saya akan gerak pada satu november dua ribu dua puluh satu',
 'date': {'1/11/2021': datetime.datetime(2021, 11, 1, 0, 0)},
 'money': {}}

In [28]:
normalizer = malaya.normalize.normalizer()
normalizer.normalize('saya akan gerak pada 1/11/2021', normalize_date = False)

{'normalize': 'saya akan gerak pada 01/11/2021',
 'date': {'1/11/2021': datetime.datetime(2021, 11, 1, 0, 0)},
 'money': {}}

### Normalize time

1. if True, `pukul 2.30` -> `pukul dua tiga puluh minit`.

Simply `normalizer.normalize(string, normalize_time = True)`, default is `True`.

In [29]:
s = 'Operasi tamat sepenuhnya pada pukul 1.30 tengah hari'
normalizer = malaya.normalize.normalizer()
normalizer.normalize(s, normalize_time = True)

{'normalize': 'Operasi tamat sepenuhnya pada pukul satu tiga puluh minit tengah hari',
 'date': {},
 'money': {}}

In [30]:
s = 'Operasi tamat sepenuhnya pada pukul 1:30:50 tengah hari'
normalizer = malaya.normalize.normalizer()
normalizer.normalize(s, normalize_time = True)

{'normalize': 'Operasi tamat sepenuhnya pada pukul satu tiga puluh minit lima puluh saat tengah hari',
 'date': {'1:30:50': datetime.datetime(2022, 4, 25, 1, 30, 50)},
 'money': {}}

In [31]:
s = 'Operasi tamat sepenuhnya pada pukul 1.30 tengah hari'
normalizer = malaya.normalize.normalizer()
normalizer.normalize(s, normalize_time = False)

{'normalize': 'Operasi tamat sepenuhnya pada 01:30:00 tengah hari',
 'date': {},
 'money': {}}

### Ignore normalize money

Let say I have a text contains `RM 77 juta` and I wish to maintain it like that.

In [32]:
text = 'Suatu ketika rakyat Malaysia dikejutkan dengan kontrak pelantikan sebanyak hampir RM 77 juta setahun yang hanya terdedah apabila diasak oleh Datuk Seri Anwar Ibrahim.'

In [33]:
normalizer = malaya.normalize.normalizer()

In [34]:
normalizer.normalize(text)

{'normalize': 'Suatu ketika rakyat Malaysia dikejutkan dengan kontrak pelantikan sebanyak hampir tujuh puluh tujuh ringgit juta setahun yang hanya terdedah apabila diasak oleh Datuk Seri Anwar Ibrahim .',
 'date': {},
 'money': {'rm 77': 'RM77'}}

In [35]:
normalizer = malaya.normalize.normalizer(money = False)
normalizer.normalize(text, normalize_text = False, check_english_func = None)

{'normalize': 'Suatu ketika rakyat Malaysia dikejutkan dengan kontrak pelantikan sebanyak hampir RM tujuh puluh tujuh juta setahun yang hanya terdedah apabila diasak oleh Datuk Seri Anwar Ibrahim .',
 'date': {},
 'money': {}}

In [36]:
normalizer.normalize(text, normalize_text = False, check_english_func = None)

{'normalize': 'Suatu ketika rakyat Malaysia dikejutkan dengan kontrak pelantikan sebanyak hampir RM tujuh puluh tujuh juta setahun yang hanya terdedah apabila diasak oleh Datuk Seri Anwar Ibrahim .',
 'date': {},
 'money': {}}

### Normalizing rules

**All these rules will ignore if first letter is capital except for normalizing titles.**

#### 1. Normalize title,

```python

{
    'dr': 'Doktor',
    'yb': 'Yang Berhormat',
    'hj': 'Haji',
    'ybm': 'Yang Berhormat Mulia',
    'tyt': 'Tuan Yang Terutama',
    'yab': 'Yang Berhormat',
    'ybm': 'Yang Berhormat Mulia',
    'yabhg': 'Yang Amat Berbahagia',
    'ybhg': 'Yang Berbahagia',
    'miss': 'Cik',
}

```

In [37]:
normalizer = malaya.normalize.normalizer()

In [38]:
normalizer.normalize('Dr yahaya')

{'normalize': 'Doktor yahaya', 'date': {}, 'money': {}}

#### 2. expand `x`

In [39]:
normalizer.normalize('xtahu')

{'normalize': 'tak tahu', 'date': {}, 'money': {}}

#### 3. normalize `ke -`

In [40]:
normalizer.normalize('ke-12')

{'normalize': 'kedua belas', 'date': {}, 'money': {}}

In [41]:
normalizer.normalize('ke - 12')

{'normalize': 'kedua belas', 'date': {}, 'money': {}}

#### 4. normalize `ke - roman`

In [42]:
normalizer.normalize('ke-XXI')

{'normalize': 'kedua puluh satu', 'date': {}, 'money': {}}

In [43]:
normalizer.normalize('ke - XXI')

{'normalize': 'kedua puluh satu', 'date': {}, 'money': {}}

#### 5. normalize `NUM - NUM`

In [44]:
normalizer.normalize('2011 - 2019')

{'normalize': 'dua ribu sebelas hingga dua ribu sembilan belas',
 'date': {},
 'money': {}}

In [45]:
normalizer.normalize('2011.01-2019')

{'normalize': 'dua ribu sebelas perpuluhan kosong satu hingga dua ribu sembilan belas',
 'date': {},
 'money': {}}

#### 6. normalize `pada NUM (/ | -) NUM`

In [46]:
normalizer.normalize('pada 10/4')

{'normalize': 'pada sepuluh hari bulan empat', 'date': {}, 'money': {}}

In [47]:
normalizer.normalize('PADA 10 -4')

{'normalize': 'pada sepuluh hari bulan empat', 'date': {}, 'money': {}}

#### 7. normalize `NUM / NUM`

In [48]:
normalizer.normalize('10 /4')

{'normalize': 'sepuluh per empat', 'date': {}, 'money': {}}

#### 8. normalize `rm NUM`

In [49]:
normalizer.normalize('RM10.5')

{'normalize': 'sepuluh ringgit lima puluh sen',
 'date': {},
 'money': {'rm10.5': 'RM10.5'}}

#### 9. normalize `rm NUM sen`

In [50]:
normalizer.normalize('rm 10.5 sen')

{'normalize': 'sepuluh ringgit lima puluh sen',
 'date': {},
 'money': {'rm 10.5': 'RM10.5'}}

#### 10. normalize `NUM sen`

In [51]:
normalizer.normalize('1015 sen')

{'normalize': 'sepuluh ringgit lima belas sen',
 'date': {},
 'money': {'1015 sen': 'RM10.15'}}

#### 11. normalize money

In [52]:
normalizer.normalize('rm10.4m')

{'normalize': 'sepuluh juta empat ratus ribu ringgit',
 'date': {},
 'money': {'rm10.4m': 'RM10400000.0'}}

In [53]:
normalizer.normalize('$10.4K')

{'normalize': 'sepuluh ribu empat ratus dollar',
 'date': {},
 'money': {'$10.4k': '$10400.0'}}

#### 12. normalize cardinal

In [54]:
normalizer.normalize('123')

{'normalize': 'seratus dua puluh tiga', 'date': {}, 'money': {}}

#### 13. normalize ordinal

In [55]:
normalizer.normalize('ke123')

{'normalize': 'keseratus dua puluh tiga', 'date': {}, 'money': {}}

#### 14. normalize date / time / datetime string to datetime.datetime

In [56]:
normalizer.normalize('2 hari lepas')

{'normalize': 'dua hari lepas',
 'date': {'2 hari lalu': datetime.datetime(2022, 4, 23, 8, 16, 31, 536643)},
 'money': {}}

In [57]:
normalizer.normalize('esok')

{'normalize': 'esok',
 'date': {'esok': datetime.datetime(2022, 4, 26, 8, 16, 31, 543711)},
 'money': {}}

In [58]:
normalizer.normalize('okt 2019')

{'normalize': 'dua puluh lima oktober dua ribu sembilan belas',
 'date': {'okt 2019': datetime.datetime(2019, 10, 25, 0, 0)},
 'money': {}}

In [59]:
normalizer.normalize('2pgi')

{'normalize': 'pukul dua',
 'date': {'2AM': datetime.datetime(2022, 4, 25, 2, 0)},
 'money': {}}

In [60]:
normalizer.normalize('pukul 8 malam')

{'normalize': 'pukul kosong malam',
 'date': {'8 PM': datetime.datetime(2022, 4, 25, 20, 0)},
 'money': {}}

In [61]:
normalizer.normalize('jan 2 2019 12:01pm')

{'normalize': 'dua januari dua ribu sembilan belas pukul dua belas satu minit',
 'date': {'jan 2 2019 12:01pm': datetime.datetime(2019, 1, 2, 12, 1)},
 'money': {}}

In [62]:
normalizer.normalize('2 ptg jan 2 2019')

{'normalize': 'pukul empat belas dua januari dua ribu sembilan belas',
 'date': {'2 PM jan 2 2019': datetime.datetime(2019, 1, 2, 14, 0)},
 'money': {}}

#### 15. normalize money string to string number representation

In [63]:
normalizer.normalize('50 sen')

{'normalize': 'lima puluh sen', 'date': {}, 'money': {'50 sen': 'RM0.5'}}

In [64]:
normalizer.normalize('20.5 ringgit')

{'normalize': 'dua puluh ringgit lima puluh sen',
 'date': {},
 'money': {'20.5 ringgit': 'RM20.5'}}

In [65]:
normalizer.normalize('20m ringgit')

{'normalize': 'dua puluh juta ringgit',
 'date': {},
 'money': {'20m ringgit': 'RM20000000.0'}}

In [66]:
normalizer.normalize('22.5123334k ringgit')

{'normalize': 'dua puluh dua ribu lima ratus dua belas ringgit tiga ribu tiga ratus tiga puluh empat sen',
 'date': {},
 'money': {'22.5123334k ringgit': 'RM22512.3334'}}

#### 16. normalize date string to %d/%m/%y

In [67]:
normalizer.normalize('1 nov 2019')

{'normalize': 'satu november dua ribu sembilan belas',
 'date': {'1 nov 2019': datetime.datetime(2019, 11, 1, 0, 0)},
 'money': {}}

In [68]:
normalizer.normalize('januari 1 1996')

{'normalize': 'satu januari seribu sembilan ratus sembilan puluh enam',
 'date': {'januari 1 1996': datetime.datetime(1996, 1, 1, 0, 0)},
 'money': {}}

In [69]:
normalizer.normalize('januari 2019')

{'normalize': 'dua puluh lima januari dua ribu sembilan belas',
 'date': {'januari 2019': datetime.datetime(2019, 1, 25, 0, 0)},
 'money': {}}

#### 17. normalize time string to %H:%M:%S

In [70]:
normalizer.normalize('2pm')

{'normalize': 'pukul empat belas',
 'date': {'2pm': datetime.datetime(2022, 4, 25, 14, 0)},
 'money': {}}

In [71]:
normalizer.normalize('2:01pm')

{'normalize': 'pukul empat belas satu minit',
 'date': {'2:01pm': datetime.datetime(2022, 4, 25, 14, 1)},
 'money': {}}

In [72]:
normalizer.normalize('2AM')

{'normalize': 'pukul dua',
 'date': {'2am': datetime.datetime(2022, 4, 25, 2, 0)},
 'money': {}}

#### 18. expand repetition shortform

In [73]:
normalizer.normalize('skit2')

{'normalize': 'sakit-sakit', 'date': {}, 'money': {}}

In [74]:
normalizer.normalize('xskit2')

{'normalize': 'tak sakit-sakit', 'date': {}, 'money': {}}

In [75]:
normalizer.normalize('xjdi2')

{'normalize': 'tak jadi-jadi', 'date': {}, 'money': {}}

In [76]:
normalizer.normalize('xjdi4')

{'normalize': 'tak jadi-jadi-jadi-jadi', 'date': {}, 'money': {}}

In [77]:
normalizer.normalize('xjdi0')

{'normalize': 'tak jadi', 'date': {}, 'money': {}}

In [78]:
normalizer.normalize('xjdi')

{'normalize': 'tak jadi', 'date': {}, 'money': {}}

#### 19. normalize `NUM SI-UNIT`

In [79]:
normalizer.normalize('61.2 kg')

{'normalize': 'enam puluh satu perpuluhan dua kilogram',
 'date': {},
 'money': {}}

In [80]:
normalizer.normalize('61.2kg')

{'normalize': 'enam puluh satu perpuluhan dua kilogram',
 'date': {},
 'money': {}}

In [81]:
normalizer.normalize('61kg')

{'normalize': 'enam puluh satu kilogram', 'date': {}, 'money': {}}

In [82]:
normalizer.normalize('61ml')

{'normalize': 'enam puluh satu milliliter', 'date': {}, 'money': {}}

In [83]:
normalizer.normalize('61m')

{'normalize': 'enam puluh satu meter', 'date': {}, 'money': {}}

In [84]:
normalizer.normalize('61.3434km')

{'normalize': 'enam puluh satu perpuluhan tiga empat tiga empat kilometer',
 'date': {},
 'money': {}}

In [85]:
normalizer.normalize('61.3434c')

{'normalize': 'enam puluh satu perpuluhan tiga empat tiga empat celsius',
 'date': {},
 'money': {}}

In [86]:
normalizer.normalize('61.3434 c')

{'normalize': 'enam puluh satu perpuluhan tiga empat tiga empat celsius',
 'date': {},
 'money': {}}

#### 20. normalize `laughing` pattern

In [87]:
normalizer.normalize('dia sakai wkwkwkawkw')

{'normalize': 'dia sakai haha', 'date': {}, 'money': {}}

In [88]:
normalizer.normalize('dia sakai hhihihu')

{'normalize': 'dia sakai haha', 'date': {}, 'money': {}}

#### 21. normalize `mengeluh` pattern

In [89]:
normalizer.normalize('Haih apa lah si yusuff ni . Mama cari rupanya celah ni')

{'normalize': 'Aduh apa lah si yusuff ini . Mama cari rupanya celah ini',
 'date': {},
 'money': {}}

In [90]:
normalizer.normalize('hais sorrylah syazzz')

{'normalize': 'aduh maaf lah syazz', 'date': {}, 'money': {}}